### Expand cell width to 100% screen width

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Install libraries

In [ ]:
# !pip install langchain
# !pip install typing-inspect==0.8.0 typing_extensions==4.5.0
# !pip install pydantic==1.10.11
# # !pip install llama-cpp-python
# # !pip install tensorflow
# !pip install accelerate
# !pip install bitsandbytes
# !pip install -U tokenizers
# !pip install transformers
# !pip install sentence-transformers

In [ ]:
!pip install ctransformers -q
!pip install langchain

### Load libraries

In [2]:
import glob
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')
from langchain.llms import CTransformers
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 

### Load configurations

In [3]:
callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])

config_sql = {'max_new_tokens': 512, 'repetition_penalty': 1.1,'temperature':0,'stop':[';']}

### Create SQLite database and load dataframe

In [4]:
file = "example.db"
df = pd.read_excel('Online Retail.xlsx',nrows=100)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


### Convert dataframe to a SQLite table in the database

In [5]:
try: 
    conn = sqlite3.connect(file) 
    print(f"Database {file} formed.") 
except: 
    print(f"Database {file} not formed.")

with sqlite3.connect(file) as conn:
    try:
        df.to_sql('Online_Retail',conn,if_exists='replace',index=False)
        print('Online_Retail table created')
    except:
        print('Online_Retail table not created')
        
con = sqlite3.connect("example.db")
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

Database example.db formed.
Online_Retail table created
[('df',), ('Online_Retail',)]


### Load Quantized LLM from cache

In [6]:
llm_from_cache = CTransformers(model="TheBloke/OpenOrca-Platypus2-13B-GGUF", 
                        model_file="openorca-platypus2-13b.q4_K_M.gguf", 
                        model_type="llama", 
                        config=config_sql,callback_manager=callback_manager)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

### Load Quantized LLM from location

In [ ]:
model_location = r'C:\Users\Sarathbabu\Downloads\openorca-platypus2-13b.Q4_K_M.gguf'
llm_from_location = CTransformers(model=model_location, 
                    config=config_sql,
                    callback_manager=callback_manager
                   )

### Context + Question

In [7]:
context = """"Context:
Online_Retail is a dataframe which contains columns such as InvoiceNo,StockCode,Description,
Quantity,InvoiceDate,UnitPrice,CustomerID,Country.
"""
question = "How many rows are there?"

### Prompt

In [8]:
prompt=f"""
{context}
Question: Write a sql query for the question '{question}'
Answer:
"""
print(prompt)


"Context:
Online_Retail is a dataframe which contains columns such as InvoiceNo,StockCode,Description,
Quantity,InvoiceDate,UnitPrice,CustomerID,Country.

Question: Write a sql query for the question 'How many rows are there?'
Answer:



### Natural language to SQL query

In [9]:
def generate_sql_query(prompt):
    model = llm_from_cache
    return model.invoke(prompt)

### LLM answer for the prompt

In [10]:
query_result = generate_sql_query(prompt)

SELECT COUNT(*) FROM Online_Retail

### Use generated query on the sqlite database

In [11]:
file = "example.db"
with sqlite3.connect(file) as conn:
    df_result = pd.read_sql_query(query_result, conn)
df_result

,COUNT(*)
0,100


### Load Quantized LLM with CTransformers Automodel from cache

In [12]:
from ctransformers import AutoModelForCausalLM,AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenOrca-Platypus2-13B-GGUF", 
                        model_file="openorca-platypus2-13b.q4_K_M.gguf", 
                        model_type="llama")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

### Context + Question

In [13]:
context = """"Context:
Online_Retail is a dataframe which contains columns such as InvoiceNo,StockCode,Description,
Quantity,InvoiceDate,UnitPrice,CustomerID,Country.
"""
question = "How many rows are there?"

### Prompt

In [14]:
prompt=f"""
{context}
Question: Write a sql query for the question '{question}'
Answer:
"""
print(prompt)


"Context:
Online_Retail is a dataframe which contains columns such as InvoiceNo,StockCode,Description,
Quantity,InvoiceDate,UnitPrice,CustomerID,Country.

Question: Write a sql query for the question 'How many rows are there?'
Answer:



### Total tokens for the model

In [15]:
prompt_tokens = len(model.tokenize(prompt))
print(f'Prompt Tokens = {prompt_tokens}')
# total_tokens = prompt_tokens + output_tokens 
# total_tokens = 512 limit for this model

Prompt Tokens = 68


### Load configurations

In [16]:
print(config_sql)

{'max_new_tokens': 512, 'repetition_penalty': 1.1, 'temperature': 0, 'stop': [';']}


### Natural language to SQL query

In [17]:
query_result=""
for text in model(prompt,max_new_tokens=512,repetition_penalty=1.1,temperature=0,stop=[';'],stream=True):
    query_result+=text
    print(text, end="", flush=True)

SELECT COUNT(*) FROM Online_Retail

### Use generated query on the sqlite database

In [18]:
file = "example.db"
with sqlite3.connect(file) as conn:
    df_result = pd.read_sql_query(query_result, conn)
df_result

,COUNT(*)
0,100
